In this Jupyter notebook, we provide an example starting code for using our `pflacg` module. This is intended to serve as an instructional example and not as a proof of performance as shown in the paper.

Please read `README.md` before continuing.

In [1]:
import logging

import matplotlib.pyplot as plt
import numpy as np

import pflacg


# Configure logging
logging.basicConfig(
    level=logging.INFO,
    format="%(levelname)s :: %(asctime)s :: %(message)s",
    datefmt="%Y-%m-%d %H:%M:%S",
)

In [2]:
# Random seeding numpy for reproducbility. (Optional)
RANDOM_SEED = 1
np.random.seed(RANDOM_SEED)

---------

# Experiment Setups

In [3]:
# Instantiating exit criterion object for determine when to halt an algorithm run.
exit_criterion = pflacg.ExitCriterion(
    "SWG",
    1.0e-4,
    criterion_reference=0.0,
    max_time=60.0,
    max_iter=1000
)

In [4]:
dimension = 10
feasible_region = pflacg.ProbabilitySimplexPolytope(dimension)

In [5]:
# Constructing a random quadratic function object as described in Appendix D.
matrix = np.random.rand(dimension, dimension)
M = matrix.T.dot(matrix)
M = M + np.identity(dimension)
b = np.random.rand(dimension)
objective_function = pflacg.Quadratic(dimension, M, b)


# Constructing the probability simplex polytope object.
feasible_region = pflacg.ProbabilitySimplexPolytope(dimension)

# Constructing an initial point object (Optional)
point_initial = pflacg.Point(
    feasible_region.initial_point,
    (1,),
    (feasible_region.initial_point,)
)

# Displaying the objective function paramter for reference.
print(f"strong convexity paramter = {objective_function.Mu}")
print(f"smoothness parameter = {objective_function.L}")

strong convexity paramter = 1.0130259160224173
smoothness parameter = 26.14007114799589


----------

# Algorithms

In [6]:
# Instantiating PFLaCG algorithm object.
PFLaCG = pflacg.ParameterFreeLaCG(fw_variant="AFW", iter_sync=False, recompile_jit=True)

INFO :: 2021-02-10 17:20:29 :: Compiling argmin_quadratic_over_active_set with numba jit.
INFO :: 2021-02-10 17:20:37 :: Compiling argmin_quadratic_over_active_set with numba jit done.


In [7]:
# Instantiating other CG algorithm objects.
AFW_algorithm = pflacg.FrankWolfe("AFW", "line_search")
PFW_algorithm = pflacg.FrankWolfe("PFW", "line_search")

----------

# Running the algorithms

In [8]:
# Running PFLaCG.
PFLaCG_run = PFLaCG.run(objective_function, feasible_region, exit_criterion, point_initial)

INFO :: 2021-02-10 17:20:41 :: Running PFLaCG: iteration = 0, duration = 0.0000000000, f_val = 2.6260050690, dual_gap = 0.0000000000, SWG = 2.4152521296
INFO :: 2021-02-10 17:20:42 :: SWG target accuracy = 1.2076260647845558
INFO :: 2021-02-10 17:20:42 :: Restarting ACC
INFO :: 2021-02-10 17:20:42 :: Creating ACC process with set size 1
INFO :: 2021-02-10 17:20:42 :: Running FAFW
INFO :: 2021-02-10 17:20:42 :: FAFW returned at global_iter = 1
INFO :: 2021-02-10 17:20:42 :: Acquiring buffer
INFO :: 2021-02-10 17:20:42 :: FAFW did better
INFO :: 2021-02-10 17:20:42 :: Outputting
INFO :: 2021-02-10 17:20:42 :: Running PFLaCG: iteration = 1, duration = 0.0537793636, f_val = 2.0182694818, dual_gap = 0.0000000000, SWG = 0.9468622239
INFO :: 2021-02-10 17:20:42 :: SWG target accuracy = 0.4734311119606196
INFO :: 2021-02-10 17:20:42 :: Restarting ACC
INFO :: 2021-02-10 17:20:42 :: Creating ACC process with set size 2
INFO :: 2021-02-10 17:20:42 :: Starting ACC process
INFO :: 2021-02-10 17:20:

In [9]:
# Running other CG algorithms.
AFW_run = AFW_algorithm.run(objective_function, feasible_region, exit_criterion, point_initial)
PFW_run = PFW_algorithm.run(objective_function, feasible_region, exit_criterion, point_initial)

INFO :: 2021-02-10 17:20:50 :: Running AFW(2.4152521295691116): iteration = 0.0000000000, duration = 0.0000000000,f_val = 2.6260050690, dual_gap = 2.4152521296, strong_wolfe_gap = 2.4152521296
INFO :: 2021-02-10 17:20:50 :: Running AFW: iteration = 1, duration = 0.0010616779, f_val = 2.0182694818, dual_gap = 2.4152521296, strong_wolfe_gap = 2.4152521296
INFO :: 2021-02-10 17:20:50 :: Running AFW: iteration = 2, duration = 0.0018372536, f_val = 1.8565267847, dual_gap = 0.9468622239, strong_wolfe_gap = 0.9468622239
INFO :: 2021-02-10 17:20:50 :: Running AFW: iteration = 3, duration = 0.0029160976, f_val = 1.7422113986, dual_gap = 0.7640912821, strong_wolfe_gap = 0.7818806344
INFO :: 2021-02-10 17:20:50 :: Running AFW: iteration = 4, duration = 0.0046606064, f_val = 1.7042228068, dual_gap = 0.4422728516, strong_wolfe_gap = 0.5738091014
INFO :: 2021-02-10 17:20:50 :: Running AFW: iteration = 5, duration = 0.0054681301, f_val = 1.6905340733, dual_gap = 0.2366945064, strong_wolfe_gap = 0.401

-----------
# Displaying the run results

The run results are returned as a list of tuples which each tuple represents `(iteration, duration, f_val, wolfe_gap, strong_wolfe_gap)`. `wolfe_gap` and `strong_wolfe_gap` are displayed if available otherwise 0.

In [10]:
print("Run result for PFLaCGL:")
for run_status in PFLaCG_run:
    print(run_status)

Run result for PFLaCGL:
(0, 0.0, 2.626005068997321, 0.0, 2.4152521295691116)
(1, 0.05377936363220215, 2.0182694818003704, 0.0, 0.9468622239212392)
(4, 0.08825182914733887, 1.7042228067751684, 0.0, 0.4012151336337366)
(6, 0.1195077896118164, 1.6784408658109393, 0.0, 0.12732975646125055)
(7, 0.1514880657196045, 1.6758077234278044, 0.0, 0.04548765169492963)
(8, 0.1824350357055664, 1.675483590953398, 0.0, 0.01732101372824113)
(10, 0.21450209617614746, 1.6754438755871621, 0.0, 0.005388484130361526)
(13, 0.24776124954223633, 1.6754385061906976, 0.0, 0.0020688586233528206)
(14, 0.2791893482208252, 1.6754380734069496, 0.0, 0.0009710245354099811)
(16, 0.31116271018981934, 1.6754378826072402, 0.0, 0.0001875432212563588)
(17, 0.3443605899810791, 1.6754378783799355, 0.0, 6.741179333324965e-05)
